In [ ]:
#Libraries mostly torch
import os
import json

import cv2
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.losses import binary_crossentropy
from keras.callbacks import Callback, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from glob import glob

%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.layers import GlobalAveragePooling2D, Dense, Conv2D, BatchNormalization, Dropout
from keras.models import Model, load_model 
import gc
!pip install segmentation-models
!pip install git+https://github.com/qubvel/segmentation_models


     |████████████████████████████████| 50 kB 4.5 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
  Cloning https://github.com/qubvel/segmentation_models to /tmp/pip-req-build-m_jij7f2
  Created wheel for segmentation-models: filename=segmentation_models-1.0.1-py3-none-any.whl size=33794 sha256=ceb73045176e783b7a1f454c741cc1954a26ea42363fc2ddb298283dd8eb1430
  Stored in directory: /tmp/pip-ephem-wheel-cache-k09k_u6p/wheels/02/cd/18/61c0bbb8766acfec68f9d20618886b7b38dfeeb95865b6ba00
Successfully built segmentation-models
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
trainImgPath = "/kaggle/input/severstal-steel-defect-detection/train_images/"
trainCsv = "/kaggle/input/severstal-steel-defect-detection/train.csv"
data=pd.read_csv(trainCsv)
data.ClassId=data.ClassId.astype(int)

train_Img_Id = []
train_class_Id = []
for i in os.listdir(trainImgPath):
    for j in range(1,5):
        train_Img_Id.append(i)
        train_class_Id.append(j)
train_Imgs = pd.DataFrame({'ImageId':train_Img_Id,'ClassId':train_class_Id})
train_Imgs.head(10)
 

,ImageId,ClassId
0,f1bd3a48a.jpg,1
1,f1bd3a48a.jpg,2
2,f1bd3a48a.jpg,3
3,f1bd3a48a.jpg,4
4,55042415c.jpg,1
5,55042415c.jpg,2
6,55042415c.jpg,3
7,55042415c.jpg,4
8,1dba99dc5.jpg,1
9,1dba99dc5.jpg,2


In [ ]:
train_data = pd.merge(train_Imgs,data ,how='outer', on=['ImageId','ClassId']) 
train_data = train_data.fillna('') 
train_data

,ImageId,ClassId,EncodedPixels
0,f1bd3a48a.jpg,1,
1,f1bd3a48a.jpg,2,
2,f1bd3a48a.jpg,3,
3,f1bd3a48a.jpg,4,
4,55042415c.jpg,1,
...,...,...,...
50267,3a9bcbdbf.jpg,4,120 27 374 29 628 31 883 32 1137 33 1392 34 16...
50268,e13e952d1.jpg,1,
50269,e13e952d1.jpg,2,
50270,e13e952d1.jpg,3,24272 5 24528 13 24784 21 25040 29 25296 34 25...


In [ ]:
train_data = pd.pivot_table(train_data, values='EncodedPixels', index='ImageId',columns='ClassId', aggfunc=np.sum).astype(str)
train_data = train_data.reset_index() # add Index column to one level with classID   
train_data.columns = ['ImageId','Defect_1','Defect_2','Defect_3','Defect_4'] 
has_defect = []
stratify = []
for index,row in train_data.iterrows():
    if row.Defect_1 or row.Defect_2 or row.Defect_3 or row.Defect_4: 
        has_defect.append(1)
    else:
        has_defect.append(0) 
    if row.Defect_1 != '':
        stratify.append(1)
    elif row.Defect_2 != '':
        stratify.append(2)
    elif row.Defect_3:
        stratify.append(3)
    elif row.Defect_4:
        stratify.append(4)
    else:
        stratify.append(0)
        
train_data["has_defect"] = has_defect 
train_data["stratify"] = stratify 

train_data.head(5) 

,ImageId,Defect_1,Defect_2,Defect_3,Defect_4,has_defect,stratify
0,0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,,,,1,1
1,00031f466.jpg,,,,,0,0
2,000418bfc.jpg,,,,,0,0
3,000789191.jpg,,,,,0,0
4,0007a71bf.jpg,,,18661 28 18863 82 19091 110 19347 110 19603 11...,,1,3


In [ ]:
WIDTH=288
HEIGHT=288
TRAINING_SIZE=7095

x_train, x_test = train_test_split(train_data, test_size = 0.1, stratify=train_data['stratify'], random_state=42)
x_train, x_val = train_test_split(x_train, test_size = 0.2, stratify = x_train['stratify'], random_state=42)
print(x_train.shape, x_val.shape, x_test.shape) 

(9048, 7) (2263, 7) (1257, 7)


In [ ]:
x_train_classification = x_train[['ImageId','has_defect']]
x_val_classification = x_val[['ImageId','has_defect']]
x_test_classification = x_test[['ImageId','has_defect']] 
print(x_train_classification.shape , x_val_classification.shape,x_test_classification.shape)
x_train_classification.head()

(9048, 2) (2263, 2) (1257, 2)


,ImageId,has_defect
1439,1d00226a0.jpg,1
8731,b10c5bf03.jpg,0
10679,d869026cd.jpg,1
5697,732bd67ee.jpg,1
3747,4c255637f.jpg,0


In [ ]:
from keras.preprocessing.image import ImageDataGenerator 
train_datagen = ImageDataGenerator(rescale=1./255., shear_range=0.2, zoom_range=0.05, rotation_range=5,
                           width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data_generator = train_datagen.flow_from_dataframe(
        dataframe=x_train_classification.astype(str),
        directory=trainImgPath,
        x_col="ImageId",
        y_col="has_defect",
        target_size=(WIDTH,HEIGHT),
        batch_size=16,
        class_mode='binary') 

valid_data_generator = test_datagen.flow_from_dataframe(
        dataframe=x_val_classification.astype(str),
        directory=trainImgPath,
        x_col="ImageId",
        y_col="has_defect",
        target_size=(WIDTH,HEIGHT),
        batch_size=16,
        class_mode='binary') 

Found 9048 validated image filenames belonging to 2 classes.
Found 2263 validated image filenames belonging to 2 classes.


In [ ]:
Classification_Model = keras.applications.xception.Xception(include_top = False, input_shape = (HEIGHT,WIDTH,3))

layer = Classification_Model.output
layer = GlobalAveragePooling2D()(layer)

layer = Dense(1024, activation='relu')(layer)
layer = BatchNormalization()(layer)
layer = Dropout(0.3)(layer)

layer = Dense(512, activation='relu')(layer)
layer = BatchNormalization()(layer)
layer = Dropout(0.3)(layer)

layer = Dense(64, activation='relu')(layer)
predictions = Dense(1, activation='sigmoid')(layer)
model = Model(inputs=Classification_Model.input, outputs=predictions)
model.summary()

83689472/83683744 [==============================] - 1s 0us/step
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 288, 288, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 143, 143, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 143, 143, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 143, 143, 32) 0           block1_conv1_bn[0][0]            
______________________

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
Training = model.fit_generator(train_data_generator, validation_data = valid_data_generator, epochs = 30, verbose=1)

Epoch 1/30
566/566 [==============================] - 327s 579ms/step - loss: 0.5928 - accuracy: 0.7083 - val_loss: 0.5913 - val_accuracy: 0.6580
Epoch 2/30
566/566 [==============================] - 284s 502ms/step - loss: 0.4547 - accuracy: 0.7905 - val_loss: 0.8697 - val_accuracy: 0.6217
Epoch 3/30
566/566 [==============================] - 285s 504ms/step - loss: 0.3574 - accuracy: 0.8500 - val_loss: 1.7428 - val_accuracy: 0.5139
Epoch 4/30
566/566 [==============================] - 286s 506ms/step - loss: 0.3331 - accuracy: 0.8594 - val_loss: 0.7232 - val_accuracy: 0.7702
Epoch 5/30
566/566 [==============================] - 288s 509ms/step - loss: 0.2925 - accuracy: 0.8789 - val_loss: 0.3763 - val_accuracy: 0.8568
Epoch 6/30
566/566 [==============================] - 288s 510ms/step - loss: 0.2957 - accuracy: 0.8783 - val_loss: 1.0431 - val_accuracy: 0.6001
Epoch 7/30
566/566 [==============================] - 288s 508ms/step - loss: 0.2843 - accuracy: 0.8842 - val_loss: 0.9528 -